In [1]:
# depth_to_lobster_converter.py
"""Quick‑use *copy‑paste* converter: runs **as‑is** in a Jupyter cell.

1. **Paste** this whole cell into a notebook.
2. It will immediately read every `*.depth` file inside
   `e:\SierraChart\Data\MarketDepthData` and write matching LOBSTER order‑book
   CSVs (10 levels each side) into `e:\SierraChart\Data\LOBSTER`.

The output CSV columns follow the original LOBSTER *orderbook* layout:
```
time, ask_price_1, ask_size_1, bid_price_1, bid_size_1, …, ask_price_10, ask_size_10, bid_price_10, bid_size_10
```
Time is **seconds since midnight UTC** (micro‑second resolution).
"""

from __future__ import annotations

import csv
import datetime as _dt
import struct
from pathlib import Path
from typing import Dict, List

# ─────────────────────────────────────────────────────────────────────────────
# Constants & helpers
# ─────────────────────────────────────────────────────────────────────────────

_SC_EPOCH = _dt.datetime(1899, 12, 30, tzinfo=_dt.timezone.utc)
_HDR_FMT = "<4I48s"            # 64‑byte header
_REC_FMT_24 = "<qBBHfII"        # 24‑byte record (SC build ≥ 2151)
_REC_FMT_20 = "<dBBHfII"        # 20‑byte record (SC build < 2151)
_REC_SIZE_24 = struct.calcsize(_REC_FMT_24)
_REC_SIZE_20 = struct.calcsize(_REC_FMT_20)
_CMD_CLEAR_BOOK = 1

_SideBook = Dict[float, int]    # price → quantity

# ─────────────────────────────────────────────────────────────────────────────
# Low‑level parser
# ─────────────────────────────────────────────────────────────────────────────

def _sc_time_to_dt(microseconds: int | float) -> _dt.datetime:
    """Convert Sierra Chart micro‑second timestamp → UTC `datetime`."""
    return _SC_EPOCH + _dt.timedelta(microseconds=microseconds)


def _iter_depth_records(path: Path):
    """Yield *(datetime, cmd, flags, price, qty)* tuples from a `.depth` file."""
    with path.open("rb") as f:
        magic, hdr_len, rec_size, version, _ = struct.unpack(_HDR_FMT, f.read(64))
        if magic != 0x44444353:
            raise ValueError(f"{path} is not a valid Sierra Chart `.depth` file")
        if rec_size == _REC_SIZE_24:
            rec_fmt, rec_size = _REC_FMT_24, _REC_SIZE_24
        elif rec_size == _REC_SIZE_20:
            rec_fmt, rec_size = _REC_FMT_20, _REC_SIZE_20
        else:
            raise ValueError(
                f"Unsupported record size {rec_size}; expected 20 or 24 bytes"
            )
        unpack_rec = struct.Struct(rec_fmt).unpack_from

        while (chunk := f.read(rec_size)):
            if len(chunk) < rec_size:
                break
            if rec_size == 24:
                ts, cmd, flags, _num_orders, price, qty, _ = unpack_rec(chunk)
            else:  # legacy build: DateTime stored as double seconds
                ts, cmd, flags, _num_orders, price, qty, _ = unpack_rec(chunk)
                ts = int(ts * 1_000_000)  # sec → µs
            yield _sc_time_to_dt(ts), cmd, flags, price, int(qty)


# ─────────────────────────────────────────────────────────────────────────────
# Snapshot helpers
# ─────────────────────────────────────────────────────────────────────────────

def _snapshot(ts: _dt.datetime, bids: _SideBook, asks: _SideBook, L: int) -> List:
    """Return a LOBSTER order‑book row with *L* levels each side."""
    midnight = ts.replace(hour=0, minute=0, second=0, microsecond=0)
    t = (ts - midnight).total_seconds()

    bids_sorted = sorted(bids.items(), key=lambda x: (-x[0]))[:L]
    asks_sorted = sorted(asks.items(), key=lambda x: (x[0]))[:L]

    bids_sorted += [(0.0, 0)] * (L - len(bids_sorted))
    asks_sorted += [(0.0, 0)] * (L - len(asks_sorted))

    row = [f"{t:.6f}"]
    for lvl in range(L):
        ap, aq = asks_sorted[lvl]
        bp, bq = bids_sorted[lvl]
        row.extend([ap, aq, bp, bq])
    return row


# ─────────────────────────────────────────────────────────────────────────────
# Core conversion logic
# ─────────────────────────────────────────────────────────────────────────────

def _convert(depth_path: Path, out_csv: Path, *, L: int = 10):
    """Convert one `.depth` file → LOBSTER CSV (internal helper)."""
    bids: _SideBook = {}
    asks: _SideBook = {}

    with out_csv.open("w", newline="") as fh:
        w = csv.writer(fh)
        header = ["time"]
        for lvl in range(L):
            header.extend([
                f"ask_price_{lvl+1}", f"ask_size_{lvl+1}",
                f"bid_price_{lvl+1}", f"bid_size_{lvl+1}",
            ])
        w.writerow(header)

        for dt, cmd, flags, price, qty in _iter_depth_records(depth_path):
            side = bids if cmd in (2, 4, 6) else asks if cmd in (3, 5, 7) else None
            if cmd == _CMD_CLEAR_BOOK:
                bids.clear(); asks.clear()
            elif side is not None:
                if cmd in (2, 3):
                    side[price] = qty
                elif cmd in (4, 5):
                    side[price] = qty
                elif cmd in (6, 7):
                    side.pop(price, None)
            w.writerow(_snapshot(dt, bids, asks, L))


# ─────────────────────────────────────────────────────────────────────────────
# HARD‑CODED EXECUTION (runs immediately when pasted in Jupyter)
# ─────────────────────────────────────────────────────────────────────────────

DEPTH_DIR = Path(r"e:\SierraChart\Data\MarketDepthData")
OUT_DIR   = Path(r"e:\SierraChart\Data\LOBSTER")
LEVELS    = 10  # depth levels each side

OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Converting *.depth in {DEPTH_DIR} → {OUT_DIR} (L={LEVELS}) …")
for depth_file in DEPTH_DIR.glob("*.depth"):
    out_name = depth_file.with_suffix("").name + f"_book{LEVELS}.csv"
    _convert(depth_file, OUT_DIR / out_name, L=LEVELS)
    print("  •", depth_file.name, "→", out_name)
print("Done.")


Converting *.depth in e:\SierraChart\Data\MarketDepthData → e:\SierraChart\Data\LOBSTER (L=10) …
  • NQH25-CME.2025-05-07.depth → NQH25-CME.2025-05-07_book10.csv
  • NQM25-CME.2025-05-07.depth → NQM25-CME.2025-05-07_book10.csv


KeyboardInterrupt: 